In [1]:
import requests
import lxml.html
import sqlite3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
import csv

In [2]:
def get_urls():
    url_list = []
    
    response = requests.get('https://world.moleg.go.kr/web/dta/lgslTrendListPage.do')
    root = lxml.html.fromstring(response.content)
    for tr in root.xpath('//*[@id="contents"]/table/tbody/tr') :
        a = tr.xpath('td[3]/a')[0]
        url = 'https://world.moleg.go.kr' + a.get('href')
        url_list.append(url)
    
    return url_list

In [3]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('  ', ' ', new_string2) # double space 제거
    new_string4 = re.sub('   ', ' ', new_string3) # double space 제거
    return new_string4

In [4]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [5]:
def get_info(url):
    details = []
    
    browser.get(url)
    title = browser.find_element('xpath','//*[@id="contents"]/div[3]').text
    country = browser.find_element('xpath','//*[@id="contents"]/div[2]/div[2]/strong').text
    upload_date = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[1]/span').text
    views = browser.find_element('xpath','//*[@id="contents"]/div[4]/ul[1]/li[2]/span').text
    trs = browser.find_element('xpath','//*[@id="lgslTrend_CTS"]').text
    
    browser.implicitly_wait(4)
    
    words = remove_punc(trs)
    details.append(title)
    details.append(country)
    details.append(upload_date)
    details.append(views)
    details.append(words)
    
    return details

In [6]:
def db_save_as_csv(data):
    data.to_csv("법제동향_page1.csv", index=False, encoding='utf-8')

In [11]:
from selenium.webdriver.chrome.options import Options

options = Options()

#바이너리 위치 설정
#options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'

browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정



urls = get_urls()
df_list = []

for url in urls:
    
    details = get_info(url)
    
    df = pd.DataFrame([{
        "제목": details[0],
        "국가": details[1],
        "등록일자": details[2],
        "조회수": details[3],
        "본문내용": details[4]
    }])
    
    df_list.append(df)
    
    browser.implicitly_wait(4)

ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)
db_save_as_csv(ARTICLE_LIST)

browser.close()
browser.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_3676\4169289942.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정
C:\Users\user\AppData\Local\Temp\ipykernel_3676\4169289942.py:8: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('C:/Users/user/.wdm/drivers/chromedriver/win32/105.0.5195/chromedriver.exe', chrome_options = options) #드라이버경로 지정


말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었던 2022 고용법개정이하 개정 고용법의 시행이 2023년 1월 1일로 연기되었다 개정 고용법은 2021년 10월25일 발의되어 2022년 3월 21일 하원 통과 3월 30일 상원을 통과하고 지난 4월 26일 왕실의 재가를 받아 9월 1일부터 시행 예정이었다 사라바난 인적자원부 장관은 상공회의소를 포함한 업계 관계자 및 이해관계자와 논의한 후 경제 회복 국면에 있는 만큼 시행에까지 시간이 필요하다는 의견을 받아들여 시행 연기를 결정하였다고 밝혔다  개정 고용법은 법정근로시간 주 45시간으로 단축 출산휴가 98일로 확대 및 배우자 출산 휴가 7일 도입 유연근무제 도입 강제노동 금지 조항 신설 고용법 적용 대상 확대 고용법 위반 벌금 인상을 주요 내용으로 하고 있다 개정 고용법은 고용주와 근로자에게 미리 준비할 시간을 제공하고 2023년 1월 1일 이후에는 더 이상의 시행 연장이 제공되지 않을 예정이다 출처 New Straits Times 202292  
마카오 법제동향  마카오 원숭이두창을 전염병 강제신고 대상으로 지정  마카오 카지노 영업중단 전후 사진 출처 로이터Reuters  2022년 7월 27일 마카오 입법회는 원숭이두창을 전염병 강제신고 의무 대상으로 지정하고 전염병 예방치료법에 근거하여 제정한 전염병 강제신고 메커니즘에 관한 시행령 별표에 원숭이두창을 추가하였다  원숭이두창monkeypox 인간 또는 동물 간에 밀접접촉을 통해서 바이러스가 감염되는 질병의 한 종류 확진되면 신체에 물집 고름 등이 나타나며 최근 사례에 근거한 치명률은 36 수준으로 확인된다세계보건기구WHO 2022  해당 시행령에 따르면 다음과 같은 상황이 발생하는 경우 반드시 신고하여야 한다  강제신고 의무 상황 감염병으로 인한 사망 감염병 집단발생 사례 및 의심 사례 감염병 관련 신고 내용에 변경사항이 있는 경우 등  마카오는 2020년 코로나19 팬데믹 발생 이후 누적 감염자 수가 상대적으로 낮은 

태국 입법동향  태국 2022년 형법전 개정법 제29권 공포  태국 정부는 2021년 5월 7일자 전자 관보 사이트에 2021년 형법전 개정법 제29권을 게재했다  이 법을 통해 형법전 제73조와 제74조가 개정되며 주요 내용은 다음과 같다  형벌 법규를 위반해도 처벌되지 않는 아동의 연령을 만10세 이하에서 만12세 이하로 상향제73조 형벌 법규를 위반해도 처벌되지는 않으나 법원의 판결에 따라 부모나 보호자 등을 소환하여 경고하거나 3년 이하의 기간을 정하여 해당 기간 내에 아동이 범죄를 저지르지 않도록 주의를 기울이도록 하고 기간 내에 범죄를 저지르는 경우에는 1회 1만바트한화 약 36만원 이하의 금액을 법원에 납부하도록 하는 등의 보호처분 대상 아동의 연령을 만10세 초과만15세 이하에서 만12세 초과만15세 이하로 조정제74조 이번 개정의 이유로는 의학 자료를 토대로 만10세 아동과 만12세 아동이 다음과 같은 측면에서 큰 차이가 없다는 점 등을 그 근거로 제시하고 있다  사고와 이성 지혜 윤리 및 잘잘못을 판단하는 면에서 발달 과정 중에 있음 아직 완전히 성숙되지 않았음 본인의 행동으로 인한 결과에 대하여 충분히 책임질 수 있는 연령이 되지 않았음 초등교육 대상 연령에 해당 교화하여 선량한 구성원으로서 사회에 복귀할 기회를 제공하여야 함 이 법은 관보에 게재된 이튿날부터 시행된다 출처 태국 정부 전자 관보 사이트 최종방문일 20220630  태국 Ch7 방송사 사이트 최종방문일 20220630 태국 Nation TV 방송사 사이트 최종방문일 20220630
아랍에미리트 입법동향  UAE 외국인 입국 및 거주에 관한 연방법 시행규칙 통과 셰이크 모하메드 빈 라시드 알 막툼 UAE 총리 겸 부통령이자 두바이 통치자는 외국인 입국 및 거주에 관한 연방법 시행규칙이 통과되었다고 밝혔다 이 시행규칙은 이상적인 생활 근로 투자처로서의 UAE 지위를 공고히 하고 전 세계에 있는 전문지식 경력 능력을 갖춘 외국의 인재를 UAE로 유치 및 초청하여 지속 가능한 국내 

In [13]:
with open(r"법제동향_page1.csv", 'r', encoding='utf-8') as f:
    
    document = pd.read_csv(f)
    
    display(document)

,제목,국가,등록일자,조회수,본문내용
0,"말레이시아, 개정 고용법 시행 연기",말레이시아,2022.09.05.,239,말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었...
1,"마카오, 원숭이두창을 《전염병 강제신고》 대상으로 지정",중국 마카오특별행정구,2022.08.01.,785,마카오 법제동향 마카오 원숭이두창을 전염병 강제신고 대상으로 지정 마카오 카지노...
2,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령,World,2022.07.22.,1593,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령 우리나라 형법제9조에 의하...
3,"독일, 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의",독일,2022.07.14.,1076,독일 입법동향 독일 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의 ...
4,"태국, 「2022년 형법전 개정법 (제29권)」 공포",태국,2022.07.12.,1104,태국 입법동향 태국 2022년 형법전 개정법 제29권 공포 태국 정부는 2021...
5,"UAE, 「외국인 입국 및 거주에 관한 연방법 시행규칙」 통과",아랍에미리트,2022.07.05.,1466,아랍에미리트 입법동향 UAE 외국인 입국 및 거주에 관한 연방법 시행규칙 통과 셰...
6,"대만, 「의료사고 예방 및 의료분쟁 처리에 관한 법률」 제정",대만,2022.07.01.,2295,대만 법제동향 대만 의료사고 예방 및 의료분쟁 처리에 관한 법률 제정 사진 출처...
7,"스페인, 소비자 보호 규정 강화",스페인,2022.06.30.,1046,스페인 법제동향 스페인 소비자 보호 규정 강화 스페인에서 지난 5월 28일 소비자...
8,미국 해운법 개정,미국,2022.06.30.,1198,미국 입법 동향 미국 해운법 개정 사진출처 Pixabay 지난 6월 16일 바이든...
9,"베트남, 7월부터 지역별 최저임금 인상",베트남,2022.06.22.,2689,베트남 입법동향 베트남 7월부터 지역별 최저임금 인상 2022년 6월 12일 베...
